In [ ]:
import numpy as np
import random
from sklearn.datasets import load_iris
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris,load_wine,load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

In [ ]:
class Node:

    def __init__(self,id=0):
        self.id = id
        self.connections = []
        self.value = random.uniform(0,1) < 0.5
        self.probabilities = []
        

    def add_connection(self,node):
        rnum = random.uniform(0,1)
        if rnum < 0.25:
            if node.id not in self.connections:
                self.connections.append(node)
                if len(self.probabilities) > 0:
                    ratio = sum(self.probabilities) / len(self.probabilities)
                    if ratio == 0:
                        ratio = 0.1
                    self.probabilities.append(ratio)
                    self.arrange_probabilities()
                else:
                    self.probabilities.append(0.1)
            else:
                print("The node is still in connections")
        else:
            pass

    def arrange_probabilities(self):
        self.probabilities = self.probabilities[0:len(self.connections)]
        s_arr = sum(self.probabilities)
        if s_arr < 1:
            self.probabilities.append(1-s_arr)
        else:
            self.probabilities = [i/s_arr for i in self.probabilities]


    def calculate_value(self):  
        if len(self.connections) >3:
            node = np.random.choice(self.connections, p=self.probabilities)
            self.value = node.value
        else:
            pass

    def print_values(self):
        print(f"the node has {len(self.connections)} number of connections")
        for node in self.connections:
            print(f"the connection is {node.id}")
            print(f"the connection's value is {node.value}")
        
        print(f"MAIN value of the node {self.id} is {self.value}")

        print(f"probability values are {self.probabilities}")

        

In [ ]:
nodes = [Node(i) for i in range(100) ]

In [ ]:
for _ in range(2000):
    node_main = random.choice(nodes)
    node_other = random.choice(nodes)
    if node_main.id == node_other.id:
        continue
    node_main.add_connection(node_other)
    
    node_main.calculate_value()

    
    

In [ ]:
node_main.print_values()


In [ ]:
class LearnerNode:

    def __init__(self, id=0,row=[],target=""):
        self.id = id
        self.nodes = []
        self.distance_val = 1
        self.value = target
        self.row  = row

    
    def select_related_nodes(self,node):
        vote = 0
        for row in self.nodes:
            if self.calculate_distance_row(row,node.row) < self.distance_val:
                vote += 1

        if vote > len(self.nodes) / 2:
            self.nodes.append(node)
        
    def calculate_distance_row(self,a,b):
        abs_dist = [abs(i-j) for i,j in zip(a,b)]  
        return sum(abs_dist)

    def assing_distance(self):
        distance_values = []
        if len(self.nodes) > 0:
            for node in self.nodes:
                distance_values.append(self.calculate_distance_row(self.row,node.row))
            
            self.distance_val = min(distance_values)


    



    

        

    

In [ ]:
data = load_iris()

In [ ]:
X = data.data
y = data.target

In [ ]:
class LearningUnits:

    def __init__(self,data,target):
        self.data = data
        self.target = target
        self.pool_of_learners = []
        


    def collect_learners(self):

        for i,row in enumerate(self.data):
            learner = LearnerNode(id=i,row=row,target=self.target[i])
            self.pool_of_learners.append(learner)
            for j in range(i):
                learner.select_related_nodes(self.pool_of_learners[j])

            learner.assing_distance()

    def selected_dist_group(self,row):
        selected_dist_group = []
        for learner in self.pool_of_learners:
            dist_temp = learner.calculate_distance_row(row,learner.row)
            if dist_temp <= learner.distance_val:
                selected_dist_group.append((dist_temp,learner))
                
        if len(selected_dist_group) > 0:
            return selected_dist_group 
        else:
            dist_per = np.inf
            for learner in self.pool_of_learners:
                dist_temp = learner.calculate_distance_row(row,learner.row)
                if dist_temp < dist_per:
                    dist_per = dist_temp
                    selected_dist_group.append((dist_temp,learner))

            return selected_dist_group[-3::]
        

    def predict(self,row):
        selected_rows = self.selected_dist_group(row)
        #print(selected_rows)
        selected_row = np.inf
        selected_value = None
        for row in selected_rows:
            if row[0] < selected_row:
                selected_row = row[0]
                selected_value = row[1].value
        return selected_value




            






In [ ]:
agent_learning = LearningUnits(data=X,target=y)
agent_learning.collect_learners()
test = X[0]
test_result = y[0]
test_val = agent_learning.predict(test)
print(test_result,test_val)

In [ ]:
def calculate_learner_accuracy(agent_learning,X,y):
    error = 0
    correct = 0
    print(X,y)
    for x_row,y_target in zip(X,y):
        prediction = agent_learning.predict(x_row)
        if prediction[0][0] == y_target:
            correct += 1
            error += 1


    print(f"correct num: {correct}, error num: {error} ratio: {correct/(error + correct)}")


In [ ]:

iris = load_iris()

X = iris.data
y = iris.target

feature_names = iris.feature_names
target_names = iris.target_names

df = pd.DataFrame(X, columns=feature_names)
df["species"] = y


model = LogisticRegression(max_iter=200)
model.fit(X, y)

y_pred = model.predict(X)

accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

print("\nConfusion Matrix:")
print(confusion_matrix(y, y_pred))

print("\nClassification Report:")
print(classification_report(y, y_pred, target_names=target_names))



## Test with Another Data

In [ ]:
data = pd.read_csv("Heart_Disease_Prediction.csv")

data.columns

In [ ]:
data.head()

In [ ]:
def calculate_learner_accuracy(agent_learning,X,y):
    error = 0
    correct = 0
    for x_row,y_target in zip(X,y):
        prediction = agent_learning.predict(x_row)
        #print(prediction,y_target)
        if prediction == y_target:
            correct += 1
        else:
            error += 1


    print(f"correct num: {correct}, error num: {error} ratio: {correct/(error + correct)}")


In [ ]:
X = data.drop('Heart Disease',axis=1).values
y = data['Heart Disease'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

In [ ]:
kmeans = KMeans(
    n_clusters=3,
    random_state=42,
    n_init=10
)

kmeans.fit(X_train,y_train)
y_pred = kmeans.predict(X_test)
y = pd.Categorical(y)


In [ ]:
def match_predict(predict,real):
    correct = 0
    error = 0
    for p,r in zip(predict,real):
        if p == r:
            correct += 1
        else:
            error+=1
    
    print(f"correct num: {correct}, error num: {error} ratio: {correct/(error + correct)}")


In [ ]:
match_predict(y_pred,y.codes)

In [ ]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)
y = pd.Categorical(y)


match_predict(y_pred,y)


# Wine DataSet Test

In [ ]:
data = load_wine()

In [ ]:


X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

In [ ]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)
y = pd.Categorical(y)


match_predict(y_pred,y)


# Kaggle Credit Score Dataset

In [ ]:
data_train = pd.read_csv('train.csv')

In [ ]:
data_train.head()

In [ ]:
data_train.info()

In [ ]:
def turn_data_into_codes(data):
    data = data.copy()
    for col in data.columns:
        if data[col].dtype == "str" or data[col].dtype ==  "object":
            print(col)
            data[col] = data[col].astype("category").cat.codes
    return data


def drop_non_float_rows(df):
    mask = df.apply(
        lambda row: pd.to_numeric(row, errors="coerce").notna().all(),
        axis=1
    )
    return df[mask]



In [ ]:
data_train = turn_data_into_codes(data_train)
data_train

data_train = drop_non_float_rows(data_train)

In [ ]:
data_train.dtypes

In [ ]:
data_train_sample = data_train.sample(300)

X = data_train_sample.drop('Credit_Score',axis=1).values
y = data_train_sample['Credit_Score'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)

match_predict(y_pred,y_test)


In [ ]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

# Breast Cancer DataSet

In [ ]:
data = load_breast_cancer()

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
agent_learning = LearningUnits(data=X_train,target=y_train)
agent_learning.collect_learners()
calculate_learner_accuracy(agent_learning,X=X_test,y=y_test)

In [ ]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)

match_predict(y_pred,y_test)

In [ ]:
rf_classifier = KMeans(n_clusters=len(np.unique(y)), random_state=0, n_init="auto")
rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)

match_predict(y_pred,y_test)